# Importing Required Modules
Below we collect the tools that we will use to build our model. After initial exploratory modeling we found that `XGBClassifier` provided the best performance, measured in terms of model accuracy.  

In [1]:
import warnings
warnings.filterwarnings('ignore')

#Basics
import pandas as pd
import numpy as np

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns

#Train Test Split
from sklearn.model_selection import train_test_split

# Imputer
from sklearn.impute import SimpleImputer

# Preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer

# Classifiers
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier

#Pipeline
from sklearn.pipeline import Pipeline

#Grid Search
from sklearn.model_selection import GridSearchCV

# Model evaluation
from sklearn.metrics import plot_confusion_matrix


# Set Random State
The random state to be used whenever a randomized process is initiated.

In [2]:
random_state = 42

# Select Columns to Drop from the Model
Provide a list of variables that should be dropped from the model. We have not observed any improvement in model performance from dropping data, measured in terms of accuracy. Training time is obviously improved by dropping columns but there seems to be a small price to pay in terms of accuracy for reducing the number of available features.

In [3]:
drop_cols = []

# Import Data
Because of the submission format requirements for the competition, it is vital that we retain the index column through out modeling so that we are able to produce predictions that can be validated using the competition's validation data.

In [4]:
features = pd.read_csv('../data/training_features.csv', index_col='id')
targets = pd.read_csv('../data/training_labels.csv', index_col='id')
df = features.join(targets, how='left')
X = df.drop('status_group', axis=1)
y = df['status_group']

# Test Train Split
For the purposes of model tuning we hold 10% of the data out for local testing.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)

# Data Validation
We experimented with both manual and automated feature selection, however neither approach improved model performance. Initially, we has issues with mixed data types in both the `public_meeting` and `permit` columns. The function below converts all categorical variables to strings to eliminate thoes errors. 

In [6]:
def convert_categorical_to_string(data):
    return pd.DataFrame(data).astype(str)

CategoricalTypeConverter = FunctionTransformer(convert_categorical_to_string)

# Classifying Variables
We will need to pre-process our data in preparation for classification. Pre-processing is different for categorical and numerical variables. In order to implement different pre-pricessing flows, we must first classify all of our variables as categorical or numerical. The function below separates columns into these two classes and excludes any variables that will be dropped from the model.

In [7]:
def classify_columns(df, drop_cols):
    """Takes a dataframe and a list of columns to drop and returns:
        - cat_cols: A list of categorical columns.
        - num_cols: A list of numerical columns.
    """
    cols = df.columns
    keep_cols = [col for col in cols if col not in drop_cols]
    cat_cols = []
    num_cols = []
    for col in keep_cols:
        if df[col].dtype == object:
            cat_cols.append(col)
        else:
            num_cols.append(col)
    return cat_cols, num_cols

In [8]:
cat_cols, num_cols = classify_columns(X_train, drop_cols)

# Building Preprocessor
Below we build a preprocessing step for our pipeline which handles all data processing.

## Categorical Pipeline
The pipeline below executes the following three steps for all of our categorical data.
1. Convert all values in categorical columns to strings. This avoids data type errors in the following steps.
2. Fill all missing values with the string `missing`.
3. One-hot encode all categorical variables. Because this data contains categorical variables with many possible values, it is possible to encounter values in testing data that was not present in the training data. For this reason, we need to set `handel_unknown` to `ignore` so that the encoder will simply ignore unknown values in testing data.

In [9]:
categorical_pipeline = Pipeline(steps=[
    ('typeConverter', CategoricalTypeConverter),
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('standardizer', OneHotEncoder(handle_unknown='ignore',dtype=float))
])

## Numerical Pipeline
The pipeline below executes two steps:
1. Imputes missing values in any numerical column with the median value from that column.
2. Scales each variable to have mean zero and standard deviation one.

In [10]:
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('standardizer', StandardScaler())
])

## Preprocessor
The column transformer below implements each of the three possible pre-processing behaviors. 
1. Apply the categorical pipeline.
2. Apply the numerical pipeline.
3. Drop the specified columns.
The if-then statement below ensures that the drop processor is only implemented if there are columns to drop. This is needed since passing an empty `drop_col` list throws an error.

In [11]:
if len(drop_cols) > 0:
    preprocessor = ColumnTransformer(
    transformers=[
        ('numericalPreprocessor', numerical_pipeline, num_cols),
        ('categoricalPreprocessor', categorical_pipeline, cat_cols),
        ('dropPreprocessor', 'drop', drop_cols)
    ])
else:
    preprocessor = ColumnTransformer(
    transformers=[
        ('numericalPreprocessor', numerical_pipeline, num_cols),
        ('categoricalPreprocessor', categorical_pipeline, cat_cols)
    ])

# Building Pipeline
Below we build our main pipeline which executes two steps.
1. Apply preprocessing to the raw data.
2. Fit a one vs rest classifier to the processed data using an eXtreme Gradient Boosted forest model.

In [12]:
pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', OneVsRestClassifier(estimator='passthrough'))
    ]
)

# Building Parameter Grid
Below we define a grid of hyper-parameters for our pipeline that will be tested in a grid search below.

In [13]:
parameter_grid = [
    {
        'classifier__estimator': [DecisionTreeClassifier()],
        'classifier__estimator__max_depth': [5],
    }]

# Instantiate Grid Search
Below we instantiate a grid search object which will fit our pipeline for every combination of the parameters defined above. Since the competition uses accuracy as it's measure of model quality, we sill evaluate model performance in terms of accuracy. For each parameter combination, the grid search will also execute five-fold cross validation. 

In order to maximize performance, we will fit our grid search on the full provided training data set and select our best hyper-parameters based on the results of cross validation. For the purposes of local model evaluation, we will then refit the best model on our local training data and use our local testing data to produce a confusion matrix.

In [14]:
grid_search = GridSearchCV(
    estimator=pipeline, 
    param_grid=parameter_grid, 
    scoring='accuracy', 
    cv=5, 
    verbose=2, 
    n_jobs=-1,
)

# Fit Grid Search
Below we fit our grid search on the full training set and select the best model hyper-parameters. This step takes an Extremely long time to run.

In [15]:
grid_search.fit(X, y)
model = grid_search.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   14.2s finished


# Display Results of Grid Search
Below we display the results of our grid search. We pay particular attention to `std_test_score` which will become larger if the model is over-fit. 

In [16]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__estimator,param_classifier__estimator__max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,6.227238,1.31484,0.502048,0.16364,DecisionTreeClassifier(max_depth=5),5,{'classifier__estimator': DecisionTreeClassifi...,0.714815,0.71835,0.721044,0.717256,0.722222,0.718737,0.002653,1


In [17]:
from sklearn.inspection import permutation_importance
r = permutation_importance(model, X, y,
                           n_repeats=5,
                           random_state=random_state,
                           n_jobs=-1
                          )

In [26]:
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{X.columns[i]:<25}: "
              f"{r.importances_mean[i]:.6f}"
              f" +/- {r.importances_std[i]:.6f}")

waterpoint_type          : 0.108844 +/- 0.001505
quantity_group           : 0.035690 +/- 0.000202
quantity                 : 0.035443 +/- 0.000094
amount_tsh               : 0.007290 +/- 0.000349
longitude                : 0.006111 +/- 0.000133
region_code              : 0.005365 +/- 0.000218
extraction_type_class    : 0.004989 +/- 0.000179
lga                      : 0.004618 +/- 0.000132
scheme_name              : 0.004102 +/- 0.000207
source_type              : 0.003934 +/- 0.000214
funder                   : 0.003238 +/- 0.000057
ward                     : 0.002295 +/- 0.000021
public_meeting           : 0.002088 +/- 0.000131
payment                  : 0.001543 +/- 0.000147
latitude                 : 0.000847 +/- 0.000092
region                   : 0.000786 +/- 0.000052
installer                : 0.000572 +/- 0.000048
extraction_type          : 0.000483 +/- 0.000042
population               : 0.000393 +/- 0.000029
source                   : 0.000185 +/- 0.000055
water_quality       